# Physics 5300 Final Project:  Lattice Vibrations

For this project, I decided to model lattice vibrations in 2D material systems. Such systems are a staple of condensed matter physics and include materials like graphene. The goal of this project is to create a fully general way to calculate lattice vibrations for an arbitrary system. I also wanted to explore more interesting phenomena like transverse (out-of-plane) waves.

The restoring forces between atoms/molecules are described by a Lenard-Jones potential. Near the potential minima, these behave like harmonic oscillator potentials. Therefore, we can model the system as a lattice of points connected by springs.

In [74]:
# Import packages

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from manim import *
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, Image

## Lagrangian Derivation:

We can describe the position of the $i^{th}$ site with the Cartesian coordinates $(x_i, y_i, z_i)$. Let us denote the equilibrium position of the site by $(x_{0_i},y_{0_i},z_{0_i})$. This defines the equilibrium length of the springs in our system.

Then, the kinetic energy terms are straightforward:

$T_i = \frac{1}{2} m_i (\dot{x_i}^2 + \dot{y_i}^2 + \dot{z_i}^2)$.

For the potential, we want to consider the contributions from displacements in all three directions and from all neighboring points $j$:

$U_i = \frac{1}{4} \sum_i k_{ij} (\sqrt{(x_i - x_j)^2 + (y_i - y_j)^2 + (z_i - z_j)^2} - l_{0_{ij}})^2$

Here, $k_{ij}$ is the spring constant for the connection between sites $i$ and $j$. The term $l_{0{ij}}\equiv\sqrt{(x_{0_i} - x_{0_j})^2 + (y_{0_i} - y_{0_j})^2 + (z_{0_i} - z_{0_j})^2}$ defines the equilibrium distance between the sites. The factor of $\frac{1}{4}$ accounts for double-counting pairs of sites.

For a 2D system ($z_{0_i} = 0$ for all sites), the small displacement limit yields no restoring force out-of-plane. Therefore, to explore transverse waves and other interesting phenomena, we will treat the system exactly, without any such small displacement approximation.

Then, the Lagrangian for the system is:

$L = \sum_i(\frac{1}{2} m_i (\dot{x_i}^2 + \dot{y_i}^2 + \dot{z_i}^2) - \frac{1}{4} \sum_i k_{ij} (\sqrt{(x_i - x_j)^2 + (y_i - y_j)^2 + (z_i - z_j)^2} - l_{0_{ij}})^2)$

Solving the Euler-Lagrange equations for this system yields the following equations of motion:

$ \ddot{x_k} = \frac{1}{2 m_k} \sum_j k_{kj}(x_k - x_j)(l_{0_{kj}}/l_{kj} - 1)$

$ \ddot{y_k} = \frac{1}{2 m_k} \sum_j k_{kj}(y_k - y_j)(l_{0_{kj}}/l_{kj} - 1)$

$ \ddot{z_k} = \frac{1}{2 m_k} \sum_j k_{kj}(z_k - z_j)(l_{0_{kj}}/l_{kj} - 1)$

Here, $l_{kj}\equiv \sqrt{(x_{k} - x_{j})^2 + (y_{k} - y_{j})^2 + (z_{k} - z_{j})^2}$ is the new distance between sites $k$ and $j$.

Thus, we have all the pieces we need to begin our numerical simulation.

## Square Lattice Initialization:

In [ ]:
# Here we initialize the system for a square lattice

# We start by creating a meshgrid for our desired points

x0_values = np.arange(-8,8,1)
y0_values = np.arange(-8,8,1)
z0_values = np.array([0])

[x0_grid, y0_grid, z0_grid] = np.meshgrid(x0_values, y0_values, z0_values)

# We reshape the array into a row vector

x0_square = x0_grid.reshape(1,-1)
y0_square = y0_grid.reshape(1,-1)
z0_square = z0_grid.reshape(1,-1)

# We define the masses of each site

m0 = 1
m_square = m0 * np.ones_like(x0_square[0])

# We subtract the transpose of the coordinates from themselves to create a matrix of the difference in that coordinate 
# and convert them to a matrix of the equilibrium distances between every pair of sites

l0_square = np.sqrt(np.power((x0_square - x0_square.T),2) + np.power((y0_square - y0_square.T),2) +
                     np.power((z0_square - z0_square.T),2))

# We can use a condition on the distances between sites to easily construct a matrix of the connectivity/spring constants
# e.g. if l0_square <= 1, this defines a nearest-neighbor square lattice
# if l0_square <= 1.5, this defines a next-nearest-neighbor square lattice (corners are connected)

k_mask_square = (l0_square > 0) & (l0_square <= 1.5)

k0_square = 1

k_square = k0_square * k_mask_square

## Honeycomb Lattice Initialization:

In [ ]:
# Here we initialize the system for a honeycomb (hexagonal) lattice

# We start by creating a meshgrid of integer linear combinations of the lattice vectors of a triangular lattice

latvec1_values = np.arange(-4,4,1)
latvec2_values = np.arange(-4,4,1)
latvec3_values = np.array([0])

[latvec1, latvec2, latvec3] = np.meshgrid(latvec1_values, latvec2_values, latvec3_values);

# We convert the meshgrid of lattice vectors to the Cartesian basis (creates a triangular lattice)
# We add a second triangular lattice that is offset from the first to produce the full honeycomb

x1 = 1/2 + (3/2 * latvec1 + 3/2 * latvec2);
y1 = -1*np.sqrt(3)*1/2 + (np.sqrt(3)/2 * latvec1 - np.sqrt(3)/2 * latvec2);
x2 = x1 + 1/2;
y2 = y1 + 1 * np.sqrt(3)/2;

# We combine the values and reshape into row vectors

x0_honeycomb = np.concatenate((x1.reshape(1, -1), x2.reshape(1, -1)), axis=1)
y0_honeycomb = np.concatenate((y1.reshape(1, -1), y2.reshape(1, -1)), axis=1)
z0_honeycomb = 0.*x0_honeycomb

# We define the masses of each site

m_honeycomb = np.ones_like(x0_honeycomb[0])

# We create a matrix of the equilibrium distances between sites

l0_honeycomb = np.sqrt(np.power((x0_honeycomb - x0_honeycomb.T),2) + np.power((y0_honeycomb - y0_honeycomb.T),2) +
                     np.power((z0_honeycomb - z0_honeycomb.T),2))

# We use a mask to impose nearest-neighbor connectivity in the lattice

k_mask_honeycomb = (l0_honeycomb > 0) & (l0_honeycomb <= 1.1)

k0_honeycomb = 1

k_honeycomb = k0_honeycomb * k_mask_honeycomb

## Lattice Class

In [ ]:
# In the lattice class, we store the values in our q vector as [x1, xdot1, y1, ydot1, z1, zdot1, x2, ...]
# These helper functions return the relevant values for a given site

def x_index(i):
        return(6*i)
    
def xdot_index(i):
        return(6*i + 1)
    
def y_index(i):
        return(6*i + 2)
    
def ydot_index(i):
        return(6*i + 3)
    
def z_index(i):
        return(6*i + 4)
    
def zdot_index(i):
        return(6*i + 5)

In [ ]:
class lattice():
    # This class solves for the motion of any arbitrary lattice given the equilibrium positions, masses, and spring 
    # We use the square lattice as a default

    def __init__(self,
                 x0 = x0_square,
                 y0 = y0_square,
                 z0 = z0_square,
                 k = k_square,
                 m = m_square):
        self.x0 = x0
        self.y0 = y0
        self.z0 = z0
        self.l0 = np.sqrt(np.power((x0 - x0.T),2) + np.power((y0 - y0.T),2) + np.power((z0 - z0.T),2))
        self.k = k
        self.m = m


    def dq_dt(self, t, q):
        """
        This function returns the derivative of the vector q 
        
        Parameters
        ----------
        t : float
            time 
        q : float
            vector with q = [x1, xdot1, y1, ydot1, z1, zdot1, x2, ...]
        """
        dq_dt_vec = np.zeros_like(q)

        # Define a variable Ns that counts through the sites
        Ns = np.arange(0,len(self.x0[0]))

        # Set the derivative of xi = xdoti, yi = ydoti, ...
        dq_dt_vec[x_index(Ns)] = q[xdot_index(Ns)]
        dq_dt_vec[y_index(Ns)] = q[ydot_index(Ns)]
        dq_dt_vec[z_index(Ns)] = q[zdot_index(Ns)]

        x = q[x_index(Ns)]
        y = q[y_index(Ns)]
        z = q[z_index(Ns)] 

        # Find the new distances between sites
        lx = x.reshape(-1,1) - x.reshape(1,-1)
        ly = y.reshape(-1,1) - y.reshape(1,-1)
        lz = z.reshape(-1,1) - z.reshape(1,-1)

        l = np.sqrt(lx**2 + ly**2 + lz**2 + 1e-12)

        # Calculate the second derivatives
        factor = (self.l0 / l - 1) * self.k

        xddot = np.sum(factor * lx, axis=1) / self.m
        yddot = np.sum(factor * ly, axis=1) / self.m
        zddot = np.sum(factor * lz, axis=1) / self.m

        dq_dt_vec[xdot_index(Ns)] = xddot
        dq_dt_vec[ydot_index(Ns)] = yddot
        dq_dt_vec[zdot_index(Ns)] = zddot

        return dq_dt_vec
            
    def solve_ode(self, 
                 ux0,
                 uy0,
                 uz0,
                 uxdot0,
                 uydot0,
                 uzdot0, 
                 t_start = 0.,
                 t_end = 10.,
                 dt = 0.1, 
                 abserr=1.0e-8, relerr=1.0e-6):
            """
            Solve the ODE given initial conditions and time interval. Here, ux0, uy0,... describe the displacements from equilibrium.
            We use solve_ivp.
            """
            # Define the initial condition vector
            q0 = np.stack([ux0 + self.x0, uxdot0, uy0 + self.y0, uydot0, uz0 + self.z0, uzdot0], axis=2).ravel()

            solution = solve_ivp(self.dq_dt, (t_start, t_end), q0, t_eval=np.arange(t_start, t_end, dt))
            return solution




## Simulations

In [73]:
# Here we simulate the square lattice by instantiating our lattice class

square = lattice()

# Define the initial displacements and velocities

ux0 = 0.*x0_square
uy0 = 0.*x0_square
uz0 = 0.*x0_square

ux0[0][(x0_square[0] == 0) & (y0_square[0] == 0)] = 0.95

uxdot0 = 0*x0_square
uydot0 = 0*x0_square
uzdot0 = 0*x0_square

# Solve the system
solution = square.solve_ode(ux0, uy0, uz0, uxdot0, uydot0, uzdot0, t_end = 20, dt=0.1)

In [ ]:
# Create a variable that counts through the sites
N_atoms = x0_square.size
Ns = np.arange(N_atoms)

# Extract the displacements from the equilibrium positions
ux = solution.y[x_index(Ns)][:] - x0_square.reshape(N_atoms, 1)
uy = solution.y[y_index(Ns)][:] - y0_square.reshape(N_atoms, 1)
uz = solution.y[z_index(Ns)][:] - z0_square.reshape(N_atoms, 1)

uxdot = solution.y[xdot_index(Ns)][:]
uydot = solution.y[ydot_index(Ns)][:]
uzdot = solution.y[zdot_index(Ns)][:]

# Calculate the magnitude of each sites displacement from equilibrium
umag = np.sqrt(np.power(ux, 2) + np.power(uy, 2) + np.power(uz, 2)).T

# Plot the state space trajectory of a single site
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(ux[10], uxdot[10])
ax.set_xlabel('$u_x$')
ax.set_ylabel('$\dot{u_x}$')

In [ ]:
# We make a simple animation to visualize the lattice vibrations

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
scat = ax.scatter([], [], [], c=[], cmap='viridis', s=50)

def init():
    ax.set_xlim([-11, 11])
    ax.set_ylim([-11, 11])
    ax.set_zlim([-2, 2])
    return scat,

def update(ti):
    x = ux[:, ti] + x0_square[0]
    y = uy[:, ti] + y0_square[0]
    z = uz[:, ti] + z0_square[0]

    u_mag = np.sqrt(ux[:, ti]**2 + uy[:, ti]**2 + uz[:, ti]**2)

    # Update the point positions
    scat._offsets3d = (x, y, z)
    
    # Update the point colors
    scat.set_array(u_mag)

    return scat,

ani = FuncAnimation(fig, update, frames=range(ux.shape[1]), init_func=init, blit=False)

# Display the animation
HTML(ani.to_jshtml())

In [ ]:
# We can do the same thing for the honeycomb lattice

honey = lattice(x0=x0_honeycomb, y0=y0_honeycomb, z0=z0_honeycomb, k=k_honeycomb, m=m_honeycomb)

ux0 = 0.*x0_honeycomb
uy0 = 0.*x0_honeycomb
uz0 = 0.*x0_honeycomb

uz0[0][90] = 1.25

uxdot0 = 0*x0_honeycomb
uydot0 = 0*x0_honeycomb
uzdot0 = 0*x0_honeycomb

solution = honey.solve_ode(ux0, uy0, uz0, uxdot0, uydot0, uzdot0, t_end = 20, dt=0.2)

In [ ]:
N_atoms = x0_honeycomb.size

Ns = np.arange(N_atoms)


ux = solution.y[x_index(Ns)][:] - x0_honeycomb.reshape(N_atoms, 1)
uy = solution.y[y_index(Ns)][:] - y0_honeycomb.reshape(N_atoms, 1)
uz = solution.y[z_index(Ns)][:] - z0_honeycomb.reshape(N_atoms, 1)

umag = np.sqrt(np.power(ux, 2) + np.power(uy, 2) + np.power(uz, 2)).T

uxdot = solution.y[xdot_index(Ns)][:]
uydot = solution.y[ydot_index(Ns)][:]
uzdot = solution.y[zdot_index(Ns)][:]

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(uz[90], uzdot[90])
ax.set_xlabel('$u_z$')
ax.set_ylabel('$\dot{u_z}$')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
scat = ax.scatter([], [], [], c=[], cmap='viridis', s=50)

def init():
    ax.set_xlim([-11, 11])
    ax.set_ylim([-11, 11])
    ax.set_zlim([-2, 2])
    return scat,

def update(ti):
    
    x = ux[:, ti] + x0_honeycomb[0]
    y = uy[:, ti] + y0_honeycomb[0]
    z = uz[:, ti] + z0_honeycomb[0]

    u_mag = np.sqrt(ux[:, ti]**2 + uy[:, ti]**2 + uz[:, ti]**2)

    # Update the point positions
    scat._offsets3d = (x, y, z)
    
    # Update the point colors
    scat.set_array(u_mag)

    return scat,

ani = FuncAnimation(fig, update, frames=range(ux.shape[1]), init_func=init, blit=False)

# Display the animation
HTML(ani.to_jshtml())

## Results

Here, we discuss a few interesting findings from the simulations. We focus on the square lattices as particular examples.

First, we compare the square lattices with nearest-neighbor and next-nearest-neighbor (corner) couplings. Below are hand-drawn illustrations of the two lattices:

Nearest-Neighbors:

![NN-lattice sketch](./square_lattice_nn_drawing.png){ width=200px }

Corners:

![Corner-lattice sketch](./square_lattice_corners_drawing.png){ width=200px }

First, we plot the state-space trajectory for a site displaced in the $x$ direction within a nearest-neighbor square lattice.

Nearest-Neighbor Square Lattice X Displacement:

![NN_x_displacement](./square_lattice_nn_x_displacement.png){ width=500px }

We see that the orbit spirals to the center as the site loses energy to its neighbors.

We can plot a similar state-space trajectory for a site in the corner square lattice.

Corner Square Lattice X Displacement:

![Corner_x_displacement](./square_lattice_corners_x_displacement.png){ width=500px }

We see that the orbit spirals at an even greater rate. This indicates that the site loses energy much quicker within this lattice.

Next, we examine the behavior of the system with a displacement in the $z$ direction. This corresponds to out-of-plane or transverse waves. We start by plotting the state-space orbit for a small initial displacement.

Corner Square Lattice Small Z Displacement:


![Corners_small_z_displacement](./square_lattice_corners_small_z_displacement.png){ width=500px }

We see that the orbit does not lose very much energy, and that approximately 1.5 'periods' were completed within the plotted time interval.

We can compare this to a plot after a large initial displacement.

Corner Square Lattice Large Z Displacement:

![Corners_large_z_displacement](./square_lattice_corners_large_z_displacement.png){ width=500px }

This time, the orbit spirals in faster and actually completes approximately 3 'periods' within the time interval. With linear systems, we expect the frequency of oscillations to be independent of their amplitude. Therefore, the behavior seen here is highly indicative of non-linearity.

## Manim Animations

In [ ]:
# Finally, we create animations using the Manim library. Below is an example of code that can be used to visualize a system

positions = []        # list of positions at each frame
for j in range(solution.t.size):
    # extract x,y,z for each atom
    xyz = np.stack([solution.y[x_index(Ns),j], solution.y[y_index(Ns),j], solution.y[z_index(Ns),j]], axis=1)
    positions.append(xyz)


eq_positions = np.stack([
     x0_honeycomb.flatten(),
     y0_honeycomb.flatten(),
     z0_honeycomb.flatten()], axis = 1)

In [ ]:
class AtomScene(ThreeDScene):
    def construct(self):
        # Create a vector group of dots at each site's position
        dots = VGroup(*[
            Dot3D(point=positions[0][i], radius=0.05, color=RED)
            for i in range(N_atoms)
        ])
        self.add(dots)

        def magnitude_to_color(mag_norm):
            # Gives the color based on the magnitude of the displacement
            cmap = plt.get_cmap("viridis")
            r, g, b, _ = cmap(mag_norm)
            return rgb_to_color([r, g, b]) 

        # ValueTracker to walk through frames
        frame_index = ValueTracker(0)

        # Updaters: on each dot, update its position and color
        def make_updater(i):
            def updater(dot):
                idx = int(frame_index.get_value())
                dot.move_to(positions[idx][i])
                color = magnitude_to_color(umag[idx][i])
                dot.set_color(color)
            return updater
        
        for i, dot in enumerate(dots):
            dot.add_updater(make_updater(i))

        # Set the camera position
        self.set_camera_orientation(phi=45 * DEGREES, theta=20 * DEGREES, distance=10)

        # Animate through all frames
        self.play(
            frame_index.animate.set_value(len(positions)-1),
            rate_func=linear,
            run_time=5  # total run time in seconds
        )
        self.wait()

## Animation Results

Below, we provide a number of animations produced with Manim

### Nearest Neighbor Square Lattice X Displacement

[▶ Nearest Neighbor Square Lattice X Displacement Animation](./square_nn_x_displacement.mp4)

### Nearest Neighbor Square Lattice Plane Wave

[▶ Nearest Neighbor Square Lattice Plane Wave Animation](./square_nn_plane_wave.mp4)


### Corner Square Lattice X Displacement

[▶ Corner Square Lattice X Displacement Animation](./square_lattice_corners_x_displacement.mp4)

### Corner Square Lattice Z Displacement

[▶ Corner Square Lattice Z Displacement Animation](./square_lattice_corners_z_displacement.mp4)

### Honeycomb Lattice X Displacement

[▶ Honeycomb Lattice X Displacement Animation](./honeycomb_lattice_x_displacement.mp4)

### Honeycomb Lattice Z Displacement

[▶ Honeycomb Lattice Z Displacement Animation](./honeycomb_lattice_z_displacement.mp4)